Area of Circle Integration ( value of pi )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

value_circle = []

N = 10000
count_inside = 0;
count_total = 0;
for i in range(N):
    x = np.random.random()
    y = np.random.random()
    if x*x + y*y < 1:
        count_inside += 1
    count_total += 1
    value_circle.append( count_inside/count_total * 4)


plt.plot(value_circle, label='Monte Carlo estimate')
plt.axhline(y=np.pi, color='r', linestyle='--', linewidth=1.5, label='π (true)')
plt.ylim(2.80, 3.50)
plt.legend()
plt.show()
print(value_circle[-1])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 10000
rng = np.random.default_rng(seed=67)
x = rng.random(N)
y = rng.random(N)

inside = (x*x + y*y) < 1  # boolean array for points indie
cum_inside = np.cumsum(inside)  # cum ???? cumulative sum based on boolean inside values
trials = np.arange(1, N+1)
value_circle = 4 * (cum_inside / trials)  # dividing inside points by total points

plt.plot(value_circle, label='Monte Carlo estimate')
plt.axhline(y=np.pi, color='r', linestyle='--', linewidth=1.5, label='π (true)')
plt.ylim(2.80, 3.50)
plt.legend()
plt.show()
print(value_circle[-1])

# Volume of N dimensional Sphere

### Riemann Sum Method

Completely Garbage Method, doesn't work for 10 dimensions for values above 4 points per dimension.

Require 1 Trillion Evaluations for 10 dimensions with 10 datapoints per dimension.

In [ ]:
def riemann_sphere_volume_iterative(dimension =10, N_per_dim=4):
    """
    Compute n-sphere volume using Riemann sum WITHOUT meshgrid.
    Iterates through grid points one at a time.
    Still impractical for dimension > 6 due to exponential number of iterations.
    """
    dx = 2.0 / N_per_dim  # Step size in each dimension
    volume_element = dx ** dimension

    # Total number of iterations needed
    total_points = N_per_dim ** dimension
    print(f"Total evaluations needed: {total_points:,}")

    inside_count = 0

    # Generate all combinations of indices iteratively
    for i in range(total_points):
        # Convert linear index to multi-dimensional coordinates
        coords = []
        temp = i
        for _ in range(dimension):
            coords.append(temp % N_per_dim)
            temp //= N_per_dim

        # Convert indices to actual coordinates in [-1, 1]
        point = np.array([(c + 0.5) * dx - 1.0 for c in coords])

        # Check if inside unit sphere
        r_squared = np.sum(point ** 2)
        if r_squared <= 1.0:
            inside_count += 1

    volume = inside_count * volume_element
    return volume

### Monte Carlo Method

In [ ]:
N = 1000000
dimension = 10

def N_dim_Sphere(N, dimension):
    rng = np.random.default_rng(seed=70)
    points = rng.random((N, dimension))

    # x^2 + y^2 .... so on needs to be done so we do this
    points = points*points
    r = np.sum(points, axis=1)
    inside = np.where(r <= 1, 1, 0)
    cum_inside_me = np.cumsum(inside)
    trials_ndim = np.arange(1, N+1)
    value_circle = (2**dimension) * (cum_inside_me/ trials_ndim)
    print(f"Monte Carlo Estimate: {value_circle[-1]:.5f}")

    true_value = (np.pi**(dimension/2)) / math.gamma(dimension/2 + 1)  # π^{d/2}/Γ(d/2+1)
    print(f"True Value: {true_value:.5f}")

    plt.figure(figsize=(12, 6))
    plt.plot(value_circle, label='Monte Carlo estimate', linewidth=1)
    plt.axhline(y=true_value, color='r', linestyle='--', linewidth=1.5, label=f'True ({true_value:.5f}) for {dimension}D-Sphere')
    plt.ylim(round(true_value-0.5, 1), round(true_value+0.5, 1))
    plt.legend()
    plt.show()




In [ ]:
N_dim_Sphere(N, dimension)
riemann_sphere_volume_iterative() # Don't increase points per dimension above 5 ( it won't run )

# Integral of function using Monte Carlo

In [ ]:
from time import time

def function(x):
    return np.exp(x)

def simpson(f, n, a, b):
    if n % 2 == 1:
        raise ValueError("n must be even for Simpson's rule")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    integer = h / 3 * (y[0] + y[-1] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]))
    return integer

def Monte_Carlo_Integration(f, x_lim: np.ndarray, y_lim: np.ndarray, N: int = 10000, seed: int = 42):
    np.random.seed(seed)
    # Start Method
    start_time = time()
    points = np.random.random((N, 2))

    # Making Random Points go into range of x_lim and y_lim
    points[:, 0] *= x_lim[1] - x_lim[0]
    points[:, 1] *= y_lim[1] - y_lim[0]
    points[:, 0] += x_lim[0]
    points[:, 1] += y_lim[0]

    f_points = f(points[:, 0]) # Array of evaluated values on given random points

    positive_region = (f_points >= 0) & (points[:, 1] >= 0) & (points[:, 1] <= f_points)
    negative_region = (f_points < 0) & (points[:, 1] < 0) & (points[:, 1] >= f_points)

    point_contributions = positive_region.astype(int) - negative_region.astype(int)

    cumulative_contributions = np.cumsum(point_contributions)

    # Running estimate at each iteration
    box_area = (x_lim[1] - x_lim[0]) * (y_lim[1] - y_lim[0])
    iterations = np.arange(1, N + 1)
    convergence = (cumulative_contributions / iterations) * box_area
    end_time = time()
    print(f"Monte Carlo Integration Complete: {end_time - start_time:.6f} seconds")

    # Analytical Answer using Simpson Method
    start_time = time()
    integral_analytical = simpson(f, N, x_lim[0], x_lim[1])
    end_time = time()
    print(f"Integral Analytical: {end_time - start_time:.6f} seconds")

    # Plotting
    plt.figure(figsize=(12, 5))
    plt.plot(iterations, convergence, label='Monte Carlo estimate', linewidth=1)
    plt.axhline(y=integral_analytical, color='r', linestyle='--', linewidth=1.5, label=f'True Value ({integral_analytical:.5f}) for Integral')
    plt.legend()
    plt.grid(True, linestyle='--')
    plt.show()

    return convergence[-1]

limits_x = np.array([0, 3*np.pi/2])
limits_y = np.array([np.exp(0), np.exp(3*np.pi/2)])
N = 10000
seed = 67
print(f"Monte Carlo: {Monte_Carlo_Integration(function, limits_x, limits_y, N, seed):.5f}")
print(f"Simpson : {simpson(function, N, limits_x[0], limits_x[1]):.5f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 1000000
x = np.random.rand(N)

y = np.arccos(1.0-x)
plt.hist(y, bins=100)
plt.show()
plt.clf()
counts, bins = np.histogram(y, bins=50)
delta = bins[1]-bins[0]
print(bins)
print(counts)
pdf = (counts/N)/delta # ( count / N ) * 1 / dy
bin_mid = (bins[:-1]+bins[1:])/2

plt.bar(bins[1:], bin_mid)
plt.show()
plt.clf()

plt.stairs(bin_mid, bins)
plt.show()
plt.clf()

plt.plot(bin_mid, pdf)
plt.plot(bin_mid, np.sin(bin_mid))
plt.show()

In [ ]:
# Closed form transformation
N = 1000000
x = np.random.rand(N)             # Uniform [0, 1)
y = np.arccos(1 - x)              # Inverse transform sampling
plt.hist(y, bins=100, density=True, alpha=0.6, label='Sampled')
plt.plot(np.linspace(0, np.pi/2, 100), np.sin(np.linspace(0, np.pi/2, 100))/1, 'r-', lw=2, label='sin(y)')
plt.legend(); plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 100000

x_1 = np.random.rand(N)
x_2 = np.random.rand(N)

r = np.sqrt(-2*np.log(x_1))
theta = 2*np.pi*x_2

y_1 = r * np.cos(theta)
y_2 = r * np.sin(theta)

normal = lambda x: np.exp(-x**2/2)/(np.sqrt(2*np.pi))
compare = np.linspace(-3, 3, N)

counts1, bins1 = np.histogram(y_1, bins=50)
deltax1 = bins1[1]-bins1[0]
pdf1 = (counts1/N)/deltax1
bin_mid1 = (bins1[:-1]+bins1[1:])/2

counts2, bins2 = np.histogram(y_2, bins=50)
deltax2 = bins2[1]-bins2[0]
pdf2 = (counts2/N)/deltax2
bin_mid2 = (bins2[:-1]+bins2[1:])/2

plt.hist(y_1, bins=50, alpha=0.5, label='cos', color='r')
plt.hist(y_2, bins=50, alpha=0.5, label='sin', color='g')
plt.legend()
plt.show()
plt.clf()

plt.plot(bin_mid1, pdf1, label='cos', color='r', alpha = 0.5)
plt.plot(compare, normal(compare), label='normal', color='b')
plt.plot(bin_mid2, pdf2, label='sin', color='g', alpha = 0.5)
plt.plot(compare, normal(compare), label='normal', color='b')
plt.legend()
plt.show()
plt.clf()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 100000
sigma = 5
mu = 2

x_1 = np.random.rand(N)
x_2 = np.random.rand(N)

r = sigma * np.sqrt(-2 * np.log(x_1))
theta = 2 * np.pi * x_2

u = r * np.cos(theta)
v = r * np.sin(theta)

y_1 = mu + u
y_2 = mu + v

normal = lambda x: np.exp(-x ** 2 / 2) / (np.sqrt(2 * np.pi))
compare = np.linspace(-3, 3, N)

counts1, bins1 = np.histogram(y_1, bins=50)
deltax1 = bins1[1] - bins1[0]
pdf1 = (counts1 / N) / deltax1
bin_mid1 = (bins1[:-1] + bins1[1:]) / 2

counts2, bins2 = np.histogram(y_2, bins=50)
deltax2 = bins2[1] - bins2[0]
pdf2 = (counts2 / N) / deltax2
bin_mid2 = (bins2[:-1] + bins2[1:]) / 2

plt.hist(y_1, bins=50, alpha=0.5, label='cos', color='r')
plt.hist(y_2, bins=50, alpha=0.5, label='sin', color='g')
plt.legend()
plt.show()
plt.clf()

plt.plot(bin_mid1, pdf1, label='cos', color='r', alpha=0.5)
plt.plot(compare, normal(compare), label='normal', color='b')
plt.plot(bin_mid2, pdf2, label='sin', color='g', alpha=0.5)
plt.plot(compare, normal(compare), label='normal', color='b')
plt.legend()
plt.show()
plt.clf()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Initialize parameters
n_samples = 100000  # total number of samples to generate
samples = []  # list to store accepted samples
x_current = 0.5  # start point in the middle of [0,1]
sigma = 0.1  # proposal step size (standard deviation)

# Run the Metropolis algorithm
for i in range(n_samples):
    # Propose a new candidate from a normal distribution centered at current x
    x_proposed = x_current + np.random.normal(0, sigma)

    # Reflect if the proposed value goes out of [0,1] (to stay within bounds)
    if x_proposed < 0:
        x_proposed = -x_proposed
    elif x_proposed > 1:
        x_proposed = 2 - x_proposed

    # Compute acceptance ratio (unnormalized)
    p_current = - np.sin(x_current) * np.log(x_current)
    p_proposed = - np.sin(x_proposed) * np.log(x_proposed)

    # Avoid divide-by-zero errors
    if p_current == 0:
        acceptance_ratio = 1
    else:
        acceptance_ratio = p_proposed / p_current

    # Accept or reject based on Metropolis criterion
    if np.random.rand() < min(1, acceptance_ratio):
        x_current = x_proposed  # accept move

    # Store the current sample
    samples.append(x_current)

# Convert to NumPy array for analysis
samples = np.array(samples)

# Plot the sampled distribution
plt.figure(figsize=(8, 5))
plt.hist(samples, bins=100, density=True, alpha=0.6, color='skyblue', label='Sampled distribution')

# Plot the true (unnormalized) function for comparison
x = np.linspace(0.001, 0.999, 500)
norm_const = np.trapezoid(-np.sin(x)*np.log(x), x)
plt.plot(x, -np.sin(x)*np.log(x) / norm_const, 'r-', lw=2, label='True (normalized) function')
plt.xlabel('x')
plt.ylabel('Probability density')
plt.legend()
plt.title('Sampling from f(x) = -sin(x) ln(x) using Metropolis algorithm')
plt.show()
